In [1]:
import sys
sys.path.append('scripts/dialent/task1/')
sys.path.append('scripts/dialent/')
sys.path.append('scripts/')
import util

In [2]:
import numpy as np
import pandas as pd
import pymorphy2

In [3]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.neural_network import MLPClassifier

In [4]:
u_dev = util.loadAllStandard('./devset/')
u_test = util.loadAllStandard('./testset//')

In [5]:
morph = pymorphy2.MorphAnalyzer()
def get_pos(token):
    pos = morph.parse(token)[0].tag.POS
    if pos == 'NOUN':
        return pos
    return "none"

In [6]:
def ne_bin(x):
    if x == 'LOC':
        return 0
    elif x == 'LOCORG':
        return 1

In [7]:
def pos_bin(x):
    if x == 'NOUN':
        return 1
    if x == 'ADJF':
        return 2
    if x == 'PRTF':
        return 3
    if x == 'VERB':
        return 4
    if x == 'CONJ':
        return 5
    return 0

In [8]:
def get_df(u):
    df = pd.DataFrame(columns=['tokens', 'tokens_id', 'start', 'end', 'ne'])
    for i in range(len(u)):
        res = u[i].makeTokenSets()
        for j in range(len(res)):
            token = res[j].toInlineString()
            split = token.split()
            df = df.append({'tokens': split[4:], 'first_token': str(split[4].split('"')[1]), 'tokens_id': int(split[1]), 'start': int(split[2][1:-1]), \
               'end': int(split[3][:-1]), 'ne': str(split[0])}, ignore_index=True)
    return df

In [90]:
#u_dev[0].makeTokenSets()[0].toInlineString().split()[4].split('"')

['', 'италии', '']

In [9]:
df_dev = get_df(u_dev)

In [10]:
df_dev = df_dev[(df_dev['ne'] == 'LOC') | (df_dev['ne'] == 'LOCORG')]
df_dev['length'] = df_dev['end'] - df_dev['start'] + 1
df_dev['ne_bin'] = df_dev['ne'].apply(ne_bin)
df_dev['pos'] = df_dev['first_token'].apply(get_pos)
df_dev['pos_bin'] = df_dev['pos'].apply(pos_bin)

In [11]:
df_dev

,tokens,tokens_id,start,end,ne,first_token,length,ne_bin,pos,pos_bin
0,"[""италии""]",16972,17,22,LOCORG,италии,6,1,NOUN,1
2,"[""грузии""]",16974,31,36,LOCORG,грузии,6,1,NOUN,1
3,"[""италии""]",16967,106,111,LOCORG,италии,6,1,NOUN,1
4,"[""грузии""]",16968,115,120,LOCORG,грузии,6,1,NOUN,1
6,"[""грузии""]",16970,189,194,LOCORG,грузии,6,1,NOUN,1
8,"[""бенрате""]",10156,25,31,LOC,бенрате,7,0,NOUN,1
9,"[""германии""]",10251,203,210,LOC,германии,8,0,NOUN,1
10,"[""бенрате""]",10252,304,310,LOC,бенрате,7,0,NOUN,1
11,"[""дюссельдорфа""]",10253,319,330,LOC,дюссельдорфа,12,0,NOUN,1
13,"[""дюссельдорфе""]",10378,1032,1043,LOC,дюссельдорфе,12,0,NOUN,1


In [12]:
df_test = get_df(u_test)

In [13]:
df_test = df_test[(df_test['ne'] == 'LOC') | (df_test['ne'] == 'LOCORG')]
df_test['length']= df_test['end'] - df_test['start'] + 1
df_test['ne_bin'] = df_test['ne'].apply(ne_bin)
df_test['pos'] = df_test['first_token'].apply(get_pos)
df_test['pos_bin'] = df_test['pos'].apply(pos_bin)

In [14]:
df_testst

,tokens,tokens_id,start,end,ne,first_token,length,ne_bin,pos,pos_bin
3,"[""тверской""]",52828,593,600,LOC,тверской,8,0,NOUN,1
4,"[""козицкого, переулка""]",52829,604,621,LOC,козицкого,18,0,none,0
5,"[""москве""]",52830,625,630,LOC,москве,6,0,NOUN,1
10,"[""петербурге""]",52835,1663,1672,LOC,петербурге,10,0,NOUN,1
18,"[""ковно""]",52843,2506,2510,LOC,ковно,5,0,none,0
20,"[""киеве""]",52845,2533,2537,LOC,киеве,5,0,NOUN,1
29,"[""ленинграде""]",52854,2960,2969,LOC,ленинграде,10,0,NOUN,1
43,"[""подоле""]",52867,4108,4113,LOC,подоле,6,0,NOUN,1
45,"[""киевском""]",52869,4099,4106,LOC,киевском,8,0,none,0
46,"[""контрактовом, зале""]",52870,4078,4094,LOC,контрактовом,17,0,none,0


In [15]:
Y_dev = df_dev['ne_bin']
Y_test = df_test['ne_bin']

In [16]:
X_dev = df_dev.drop(labels=['tokens', 'ne', 'ne_bin','first_token', 'pos'], axis=1)
X_test = df_test.drop(labels=['tokens', 'ne', 'ne_bin','first_token', 'pos'], axis=1)

In [17]:
clf = RandomForestClassifier()

In [22]:
clf = MLPClassifier()

In [23]:
clf.fit(X_dev, Y_dev)

MLPClassifier(activation='relu', alpha=0.0001, batch_size='auto', beta_1=0.9,
       beta_2=0.999, early_stopping=False, epsilon=1e-08,
       hidden_layer_sizes=(100,), learning_rate='constant',
       learning_rate_init=0.001, max_iter=200, momentum=0.9,
       nesterovs_momentum=True, power_t=0.5, random_state=None,
       shuffle=True, solver='adam', tol=0.0001, validation_fraction=0.1,
       verbose=False, warm_start=False)

In [24]:
predict = clf.predict(X_test)

In [25]:
ans = np.array(Y_test)

In [26]:
a = 0
for i in range(len(predict)):
    if predict[i] == ans[i]:
        a += 1
print(a / len(predict))

0.4625158831003812


In [6]:
u = util.loadAllStandard('./testset/')
for i in range(len(u)):
    res = u[i].makeTokenSets()
    row = []
    for j in range(len(res)):
        token = res[j].toInlineString()
        if token[0] == 'L':
            #Заменить в этом ифе код на код предсказывания
            split = token.split()
            start = int(split[2][1:-1])
            end = int(split[3][:-1])
            row.append('%s %d %d\n' % (split[0], start, end-start+1))
        else:
            split = token.split()
            start = int(split[2][1:-1])
            end = int(split[3][:-1])
            row.append('%s %d %d\n' % (split[0], start, end-start+1))
    with open('./result/' + u[i].name + '.task1', 'w') as f:
        f.writelines(row)        

In [7]:
!python scripts/t1_eval.py -s ./testset -t ./result -l

book_3954
Type    P        R        F1       TP1      TP2      In Std.  In Test.
per        0.9993   0.9993   0.9993  1342.00  1342.00     1343     1343
loc        0.9933   0.9933   0.9933  1220.76  1220.76     1229     1229
org        0.9895   0.9895   0.9895  1557.55  1557.55     1574     1574
overall    0.9938   0.9938   0.9938  4112.31  4112.31     4138     4138
